In [195]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score

In [2]:
ATAC_seq = pd.read_csv('data/Hackathon2024.ATAC.txt.gz', sep='\t', index_col=0, compression='gzip')
RNA_seq = pd.read_csv('data/Hackathon2024.RNA.txt.gz', sep='\t', index_col=0, compression='gzip')
meta = pd.read_csv('data/Hackathon2024.Meta.txt.gz', sep='\t', compression='gzip')
testing_set = pd.read_csv('data/Hackathon2024.Testing.Set.Peak2Gene.Pairs.txt.gz', sep='\t', compression='gzip')
training_set = pd.read_csv('data/Hackathon2024.Training.Set.Peak2Gene.Pairs.txt.gz', sep='\t', compression='gzip')

In [248]:
## This corrected version fits the TruncatedSVD model to the ATAC-seq and RNA-seq data separately, vectors
## of length 1000 that represent each gene and peak

svd_rna = TruncatedSVD(n_components=1000, random_state=42)
rna_transformed = svd_rna.fit_transform(RNA_seq)

In [249]:
svd_atac = TruncatedSVD(n_components=1000, random_state=42)
atac_transformed = svd_atac.fit_transform(ATAC_seq)

In [250]:
## put the transformed vectors into dataframes so genes and peaks can be easily indexed
RNA_seq_transformed = pd.DataFrame(data=rna_transformed, index=RNA_seq.index)
ATAC_seq_transformed = pd.DataFrame(data=atac_transformed, index=ATAC_seq.index)

In [270]:
## get the vectors for a list of pairs like chr1-4i9u902u_AKT1
## the vectors are now combined through subtraction, but we can proabably find a more effective way to combine them
def get_vecs(pairs: list[str]):
    genes = [pair.split('_')[1] for pair in pairs]
    chrs = [pair.split('_')[0] for pair in pairs]
    return RNA_seq_transformed.loc[genes].values - ATAC_seq_transformed.loc[chrs].values 

In [271]:
training_sample = training_set.sample(150)
training_sample_test = training_set.iloc[list(set(range(300)).difference(training_sample.index.values))]

y_train = list(training_sample['Peak2Gene'].astype(int))
x_train = get_vecs(list(training_sample['Pair']))

x_train_test = get_vecs(list(training_sample_test['Pair']))
y_train_test = list(training_sample_test['Peak2Gene'].astype(int))

In [276]:
clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=[500, 100, 2], random_state=1, max_iter=1000)
clf.fit(x_train, y_train)
clf.score(x_train_test, y_train_test)

0.9933333333333333

## Retrain model on all training data and create predictions.csv

In [277]:
y_train = list(training_set['Peak2Gene'].astype(int))
x_train = get_vecs(list(training_set['Pair']))

x_test = get_vecs(list(testing_set['Pair']))

In [279]:
clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=[500, 100, 2], random_state=1, max_iter=1000)
clf.fit(x_train, y_train)

testing_set['Peak2Gene'] = clf.predict(x_test).astype(bool)

In [280]:
# format for R
testing_set['Peak2Gene'] = testing_set['Peak2Gene'].map(lambda x: 'TRUE' if x  else 'FALSE')
testing_set.set_index('peak', drop=True).to_csv('predictions.csv')